# 🚀 Analyse Rhétorique Collaborative par Agents IA - Orchestrateur Principal (v_py)

**Objectif:** Ce notebook orchestre et exécute une analyse rhétorique multi-agents sur un texte donné, en utilisant une structure de projet Python modulaire.

**Structure Modulaire Utilisée:**
* `config/`: Fichiers de configuration (`.env`).
* `core/`: Composants partagés (État, StateManager, Stratégies, Setup JVM & LLM).
* `utils/`: Fonctions utilitaires.
* `ui/`: Logique de l'interface utilisateur (`text_input_ui.py`) et lanceur (`run_ui_configuration.ipynb` - optionnel).
* `agents/`: Définitions des agents spécialisés (PM, Informal, PL).
* `orchestration/`: Logique d'exécution de la conversation (`analysis_runner.py`).

**Flux d'Exécution:**
1.  Chargement de l'environnement (`.env`).
2.  Configuration du Logging.
3.  Initialisation de la JVM (via `core.jvm_setup`).
4.  Création du service LLM (via `core.llm_service`).
5.  Affichage de l'UI de configuration (via `ui.text_input_ui`) pour obtenir le texte.
6.  Exécution de l'analyse collaborative (via `orchestration.analysis_runner`) si un texte est fourni.
7.  Affichage des résultats (logs, état final).

**Prérequis:**
* Un fichier `.env` à la racine contenant les clés API, configurations LLM, et la phrase secrète `TEXT_CONFIG_PASSPHRASE`.
* Un environnement Java Development Kit (JDK >= 11) correctement installé et configuré (`JAVA_HOME`).
* Les dépendances Python installées (voir `requirements.txt` ou `pyproject.toml` du projet).
* Les JARs Tweety placés dans le dossier `libs/`.
* Le fichier `extract_sources.json.gz.enc` (s'il existe déjà) dans `data/`.

## 1. Chargement de l'Environnement (.env)

In [1]:
from dotenv import load_dotenv, find_dotenv
import os

loaded = load_dotenv(find_dotenv(), override=True)
print(f".env chargé: {loaded}")

# Vérification rapide de quelques variables clés (optionnel)
print(f"LLM Model ID présent: {'OPENAI_CHAT_MODEL_ID' in os.environ}")
print(f"LLM API Key présent: {'OPENAI_API_KEY' in os.environ}")
print(f"UI Passphrase présente: {'TEXT_CONFIG_PASSPHRASE' in os.environ}")

.env chargé: True
LLM Model ID présent: True
LLM API Key présent: True
UI Passphrase présente: True


## 2. Configuration du Logging Global

In [2]:
import logging

# Configuration de base - Les modules peuvent définir des loggers plus spécifiques
logging.basicConfig(
    level=logging.INFO, # Mettre DEBUG pour voir plus de détails
    format='%(asctime)s [%(levelname)s] [%(name)s] %(message)s',
    datefmt='%H:%M:%S'
)

# Réduire la verbosité de certaines bibliothèques
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel.connectors.ai").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel.kernel").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel.functions").setLevel(logging.WARNING)
# Garder INFO pour l'orchestration et les agents
logging.getLogger("Orchestration").setLevel(logging.INFO)
logging.getLogger("semantic_kernel.agents").setLevel(logging.INFO)
logging.getLogger("App.UI").setLevel(logging.INFO) # Logger pour l'UI

logging.info("Logging configuré.")

15:47:24 [INFO] [root] Logging configuré.


## 3. Initialisation de la JVM

In [3]:
import logging
from core.jvm_setup import initialize_jvm

logging.info("Tentative d'initialisation de la JVM...")
# La fonction initialize_jvm gère maintenant aussi le téléchargement des JARs
jvm_ready_status = initialize_jvm(lib_dir_path="libs")

if jvm_ready_status:
    logging.info("✅ JVM initialisée avec succès ou déjà active.")
else:
    logging.warning("⚠️ JVM n'a pas pu être initialisée. L'agent PropositionalLogicAgent ne fonctionnera pas.")

15:47:24 [INFO] [root] Tentative d'initialisation de la JVM...
15:47:24 [INFO] [Orchestration.JPype] 
--- Préparation et Initialisation de la JVM via JPype ---
15:47:24 [INFO] [Orchestration.JPype] 
--- Vérification/Téléchargement des JARs Tweety v1.28 ---
15:47:24 [INFO] [Orchestration.JPype] Vérification de l'accès à https://tweetyproject.org/builds/1.28/...
15:47:25 [INFO] [Orchestration.JPype] ✔️ URL de base Tweety v1.28 accessible.
15:47:25 [INFO] [Orchestration.JPype] 
--- Vérification/Téléchargement JAR Core ---
15:47:25 [INFO] [Orchestration.JPype] ✔️ JAR Core 'org.tweetyproject.tweety-full-1.28-with-dependencies.jar': déjà présent.
15:47:25 [INFO] [Orchestration.JPype] 
--- Vérification/Téléchargement des 32 JARs de modules ---


Modules JARs:   0%|          | 0/32 [00:00<?, ?it/s]

15:47:25 [INFO] [Orchestration.JPype] -> Modules: 0 téléchargés, 32/32 présents.
15:47:25 [INFO] [Orchestration.JPype] 
--- Vérification/Téléchargement des 3 binaires natifs (Windows) ---


Binaires Natifs:   0%|          | 0/3 [00:00<?, ?it/s]

15:47:25 [INFO] [Orchestration.JPype] -> Binaires natifs: 0 téléchargés, 3/3 présents.
15:47:25 [INFO] [Orchestration.JPype]    Note: S'assurer que le chemin 'C:\dev\2025-Epita-Intelligence-Symbolique\argumentation_analysis\libs\native' est inclus dans java.library.path lors du démarrage JVM.
15:47:25 [INFO] [Orchestration.JPype] --- Fin Vérification/Téléchargement Tweety ---
15:47:25 [INFO] [Orchestration.JPype] ℹ️ Variable d'environnement JAVA_HOME non définie.
15:47:25 [INFO] [Orchestration.JPype] ℹ️ Tentative de détection via heuristiques spécifiques à l'OS...
15:47:25 [INFO] [Orchestration.JPype]   1 installations Java potentielles trouvées par heuristique.
15:47:25 [INFO] [Orchestration.JPype] ✔️ Répertoire Java Home valide trouvé via heuristique: C:\Program Files\Java\jdk-21
15:47:25 [INFO] [Orchestration.JPype] ✅ JAVA_HOME défini dynamiquement à 'C:\Program Files\Java\jdk-21' pour cette session.
15:47:25 [INFO] [Orchestration.JPype] ⏳ Tentative de démarrage JVM...
15:47:25 [INF

## 4. Création du Service LLM

In [4]:
import logging
from core.llm_service import create_llm_service

llm_service = None
try:
    logging.info("Création du service LLM...")
    llm_service = create_llm_service() # Utilise l'ID par défaut
    logging.info(f"✅ Service LLM créé avec succès (ID: {llm_service.service_id}).")
except Exception as e:
    logging.critical(f"❌ Échec critique de la création du service LLM: {e}", exc_info=True)
    print(f"❌ ERREUR: Impossible de créer le service LLM. Vérifiez la configuration .env et les logs.")
    # raise # Décommenter pour arrêter si LLM indispensable

15:47:34 [INFO] [root] Création du service LLM...
15:47:34 [INFO] [Orchestration.LLM] --- Configuration du Service LLM (global_llm_service) ---
15:47:34 [INFO] [Orchestration.LLM] Configuration Service: OpenAIChatCompletion...
15:47:35 [INFO] [Orchestration.LLM] Service LLM OpenAI (gpt-4o-mini) créé avec ID 'global_llm_service'.
15:47:35 [INFO] [root] ✅ Service LLM créé avec succès (ID: global_llm_service).


## 5. Configuration de la Tâche via l'Interface Utilisateur

La cellule suivante importe et exécute la fonction `configure_analysis_task` du module `ui.app`.

Cela affichera l'interface utilisateur. Interagissez avec l'UI (sélectionnez une source, préparez/extrayez le texte si nécessaire) puis cliquez sur **"Lancer l'Analyse"**. Le texte préparé sera retourné et stocké pour l'étape suivante.

**La cellule attendra la fin de votre interaction avec l'UI.**

In [5]:
import logging
import traceback

texte_pour_analyse = None # Initialiser

try:
    # Importer la fonction UI depuis le module .py
    # Assurez-vous que ce notebook est exécuté depuis la racine du projet
    from ui.app import configure_analysis_task
    logging.info("Fonction 'configure_analysis_task' importée depuis ui.app.")

    # Appeler la fonction pour afficher l'UI et obtenir le texte
    logging.info("Lancement de l'interface de configuration...")
    texte_pour_analyse = configure_analysis_task() # Bloque jusqu'au clic sur "Lancer"

except ImportError as e_import:
    logging.critical(f"❌ ERREUR: Impossible d'importer l'UI: {e_import}")
    print(f"❌ ERREUR d'importation de l'interface utilisateur: {e_import}. Vérifiez la structure du projet et les __init__.py.")
except Exception as e_ui:
    logging.error(f"❌ Une erreur est survenue lors de l'exécution de l'interface utilisateur : {e_ui}", exc_info=True)
    print(f"❌ Une erreur est survenue pendant l'exécution de l'UI : {e_ui}")
    traceback.print_exc()

# Vérifier si on a bien reçu du texte après l'interaction UI
if not texte_pour_analyse:
    logging.warning("\n❌ Aucun texte préparé via l'UI. L'analyse ne peut pas continuer.")
    print("\n❌ Aucun texte préparé via l'UI. L'analyse ne peut pas continuer.")
else:
    logging.info(f"\n✅ Texte prêt pour l'analyse (longueur: {len(texte_pour_analyse)}) récupéré via l'UI.")
    print(f"\n✅ Texte prêt pour l'analyse (longueur: {len(texte_pour_analyse)}). Passage à l'exécution.")
    # print("--- Extrait Texte --- \n", texte_pour_analyse[:500] + "...") # Décommenter pour voir extrait

15:47:35 [INFO] [App.UI.Config] Vérification de la phrase secrète 'TEXT_CONFIG_PASSPHRASE' dans .env...
15:47:35 [INFO] [App.UI.Config] ✅ Phrase secrète trouvée. Dérivation de la clé...
15:47:35 [INFO] [App.UI.Config] ✅ Clé de chiffrement dérivée et encodée.
15:47:35 [INFO] [App.UI.Config] Cache répertoire assuré : C:\dev\2025-Epita-Intelligence-Symbolique\argumentation_analysis\text_cache
15:47:35 [INFO] [App.UI.Config] Répertoire config UI assuré : C:\dev\2025-Epita-Intelligence-Symbolique\argumentation_analysis\data
15:47:35 [INFO] [App.UI.Config] Répertoire temporaire assuré : C:\dev\2025-Epita-Intelligence-Symbolique\argumentation_analysis\temp_downloads
15:47:35 [INFO] [App.UI.Config] ✅ Configuration chargée depuis extract_sources.json
15:47:35 [INFO] [App.UI.Config] Config UI initialisée. 4 sources chargées.
15:47:35 [INFO] [App.UI.Utils] Fonctions utilitaires UI définies.
15:47:35 [CRITICAL] [root] ❌ ERREUR: Impossible d'importer l'UI: cannot import name 'CONFIG_FILE' from 'ui.

❌ ERREUR d'importation de l'interface utilisateur: cannot import name 'CONFIG_FILE' from 'ui.config' (c:\dev\2025-Epita-Intelligence-Symbolique\argumentation_analysis\ui\config.py). Vérifiez la structure du projet et les __init__.py.

❌ Aucun texte préparé via l'UI. L'analyse ne peut pas continuer.


## 6. Exécution de l'Analyse Collaborative

Si un texte a été préparé avec succès via l'UI et que le service LLM est disponible, cette cellule lance l'analyse collaborative en appelant la fonction `run_analysis_conversation`.

In [6]:
import nest_asyncio
import asyncio
import logging
import traceback

# Appliquer nest_asyncio pour compatibilité Jupyter
nest_asyncio.apply()

# Lancer seulement si on a un texte ET un service LLM valide
if texte_pour_analyse and llm_service:
    logging.info("\n🚀 Tentative de lancement de l'exécution asynchrone de l'analyse...")
    print("\n🚀 Lancement de l'analyse collaborative (peut prendre du temps)... ")
    try:
        # Importer la fonction d'orchestration
        from orchestration.analysis_runner import run_analysis_conversation

        # Exécuter la fonction d'analyse en passant le texte et le service LLM
        # asyncio.run() gère la boucle d'événements
        asyncio.run(run_analysis_conversation(
            texte_a_analyser=texte_pour_analyse,
            llm_service=llm_service
        ))

        logging.info("\n🏁 Exécution asyncio.run terminée.")
        print("\n🏁 Analyse terminée.")

    except ImportError as e_import_run:
         logging.critical(f"❌ ERREUR: Impossible d'importer 'run_analysis_conversation': {e_import_run}")
         print(f"❌ ERREUR d'importation de la fonction d'orchestration: {e_import_run}")
    except Exception as e_analysis:
        logging.error(f"\n❌ Une erreur est survenue pendant l'exécution de l'analyse : {e_analysis}", exc_info=True)
        print(f"\n❌ Une erreur est survenue pendant l'exécution de l'analyse : {e_analysis}")
        traceback.print_exc()

elif not texte_pour_analyse:
    logging.warning("Analyse non lancée : aucun texte n'a été préparé via l'UI.")
    print("\n Analyse non lancée : aucun texte n'a été préparé via l'UI.")
else: # Implique que llm_service est None ou invalide
     logging.error("Analyse non lancée : le service LLM n'a pas pu être configuré ou est invalide.")
     print("\n Analyse non lancée : le service LLM n'a pas pu être configuré.")

15:47:35 [WARNING] [root] Analyse non lancée : aucun texte n'a été préparé via l'UI.



 Analyse non lancée : aucun texte n'a été préparé via l'UI.


## 7. Résultats et Conclusion

Vérifiez les logs et l'état final JSON affichés par l'exécution précédente pour voir le résultat de l'analyse collaborative.

## 8. 🏁 Pistes d'Amélioration Futures

*(Repris des notebooks précédents)*

**Prochaines étapes possibles :**
* **Activer & Finaliser PL:** Implémenter réellement les appels JPype/Tweety dans `PropositionalLogicPlugin._internal_execute_query` et tester de bout en bout l'exécution des requêtes logiques (parsing, query, interprétation).
* **Affiner Analyse Sophismes:** Améliorer les instructions de `InformalAnalysisAgent` pour une exploration plus fine de la taxonomie (gestion de la profondeur, choix des branches) ou l'attribution de sophismes spécifiques basée sur les détails récupérés (`get_fallacy_details`).
* **Externaliser Prompts & Config:** Déplacer les prompts et configurations (ex: noms agents, constantes) hors du code Python vers des fichiers dédiés (YAML, JSON, .env) pour une meilleure maintenabilité. Utiliser `kernel.import_plugin_from_directory`.
* **Gestion Erreurs Agents:** Renforcer la capacité des agents à gérer les erreurs retournées par les outils (`FUNC_ERROR:`) et à adapter leur plan (ex: demander une clarification, réessayer, passer à autre chose).
* **Nouveaux Agents/Capacités:** Implémenter des agents pour d'autres logiques (FOL, Modale), d'autres tâches (résumé, extraction d'entités) ou d'autres outils (recherche web, base de données).
* **État RDF/KG:** Explorer le passage à une structure d'état plus riche et sémantiquement structurée en utilisant RDF/KG (avec `rdflib` ou une base de graphe) pour représenter les arguments, relations, et métadonnées de manière plus formelle.
* **Interface Utilisateur:** Créer une interface (ex: avec Gradio, Streamlit) pour faciliter l'interaction et la visualisation de l'analyse (au-delà de l'UI de configuration initiale).